In [1]:
import pandas as pd
import multiprocessing
pd.set_option('display.max_colwidth', None)
import re
import os
os.chdir(r'D:\Хакатон')
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
nltk.download('punkt')
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_score, recall_score, precision_recall_curve
from matplotlib import pyplot as plt
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LinearRegression

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sunny\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
df = pd.read_excel("ИКРБС_2020-2024_гг..xlsx")

In [3]:
df.fillna('Отсутствует', inplace=True)

In [4]:
df.drop(["№ п/п", "Регистрационный номер"],axis=1,inplace=True)

In [5]:
df['text_data'] = df['Наименование отчёта'] + ' ' + df['Реферативное описание отчета'] + ' ' + df['Ключевые слова']



In [6]:
df = df.drop_duplicates()

In [7]:
df = df.sample(frac=0.1)


In [8]:
sentence_example = df.iloc[1]["text_data"]
tokens = word_tokenize(sentence_example, language="russian")
tokens_without_punctuation = [i for i in tokens if i not in string.punctuation]
russian_stop_words = stopwords.words("russian")
tokens_without_stop_words_and_punctuation = [i for i in tokens_without_punctuation if i not in russian_stop_words]
snowball = SnowballStemmer(language="russian")
stemmed_tokens = [snowball.stem(i) for i in tokens_without_stop_words_and_punctuation]


In [9]:
print(f"Исходный текст: {sentence_example}")
print("-----------------")
print(f"Токены: {tokens}")
print("-----------------")
print(f"Токены без пунктуации: {tokens_without_punctuation}")
print("-----------------")
print(f"Токены без пунктуации и стоп слов: {tokens_without_stop_words_and_punctuation}")
print("-----------------")
print(f"Токены после стемминга: {stemmed_tokens}")
print("-----------------")

Исходный текст: Религиозные системы Востока: традиции и современность На данном этапе НИР были проведены исследования по исламу, буддизму, религиям Китая, Индии и стран Юго-Восточной Азии. Были затронуты вопросы как теоретического характера, так и конкретные исторические проблемы и тексты. Хронологические рамки исследований варьируются от древности до современности.  Результаты исследований данного этапа НИР были опубликованы в виде статей и тезисов (66 публикаций), из них 5 в журналах, входящих в базу  RSCI Web of Science, пять - Scopus, 13 в журналах из списка ВАК, 15 - РИНЦ. Было опубликовано 4 монографии, из них 1 индивидуальная, 3 - коллективных. Полученные результаты также были представлены в виде 34 докладов на российских и международных научных конференциях. Полученные результаты могут быть использованы при изучении истории стран Азии, при подготовке учебных курсов для историков, философов, регионоведов и религиоведов, а также при выработке государственной политики как в отноше

In [10]:
snowball = SnowballStemmer(language="russian")
russian_stop_words = stopwords.words("russian")

def tokenize_sentence(sentence: str, remove_stop_words: bool = True):
    tokens = word_tokenize(sentence, language="russian")
    tokens = [i for i in tokens if i not in string.punctuation]
    if remove_stop_words:
        tokens = [i for i in tokens if i not in russian_stop_words]
    tokens = [snowball.stem(i) for i in tokens]
    return tokens

In [11]:
tokenize_sentence(sentence_example)

['религиозн',
 'систем',
 'восток',
 'традиц',
 'современ',
 'на',
 'дан',
 'этап',
 'нир',
 'провед',
 'исследован',
 'ислам',
 'буддизм',
 'религ',
 'кит',
 'инд',
 'стран',
 'юго-восточн',
 'аз',
 'был',
 'затронут',
 'вопрос',
 'теоретическ',
 'характер',
 'конкретн',
 'историческ',
 'проблем',
 'текст',
 'хронологическ',
 'рамк',
 'исследован',
 'варьир',
 'древност',
 'современ',
 'результат',
 'исследован',
 'дан',
 'этап',
 'нир',
 'опубликова',
 'вид',
 'стат',
 'тезис',
 '66',
 'публикац',
 '5',
 'журнал',
 'входя',
 'баз',
 'RSCI',
 'Web',
 'of',
 'Science',
 'пят',
 'Scopus',
 '13',
 'журнал',
 'списк',
 'вак',
 '15',
 'ринц',
 'был',
 'опубликова',
 '4',
 'монограф',
 '1',
 'индивидуальн',
 '3',
 'коллективн',
 'получен',
 'результат',
 'такж',
 'представл',
 'вид',
 '34',
 'доклад',
 'российск',
 'международн',
 'научн',
 'конференц',
 'получен',
 'результат',
 'могут',
 'использова',
 'изучен',
 'истор',
 'стран',
 'аз',
 'подготовк',
 'учебн',
 'курс',
 'историк',
 'фил

In [12]:
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['text_data'].values.astype('U'))

In [13]:
y = df['ГРНТИ']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

In [15]:
model = SGDClassifier(max_iter=10000) 
model.fit(X_train, y_train)

SGDClassifier(max_iter=10000)

In [16]:
y_pred = model.predict(X_test)

In [17]:

accuracy = accuracy_score(y_test, y_pred)
print('Точность:', accuracy)

Точность: 0.11865443425076452


In [18]:
# Новый текст
new_text = "электроника"

# Преобразование нового текста в вектор признаков
new_vector = vectorizer.transform([new_text])

# Получение предсказания рубрики ГРНТИ
predicted_grnti = model.predict(new_vector)

print("Предсказанная рубрика ГРНТИ:", predicted_grnti)

Предсказанная рубрика ГРНТИ: ['83.33.23: Статистика потребления населением материальных благ и услуг; 83.33.29: Статистика заработной платы; 83.33.41: Статистика социального обеспечения']
